# OpenAI: Basic ReAct AI Agent from Scratch
* Collaborators:
    * Roberto Rodriguez @Cyb3rWard0g

In [ ]:
! pip install openai
! pip install python-dotenv
! pip install pydantic

## The LLM Client Class

In [2]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

## The Short-Term Memory Class

In [3]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""
    
    def __init__(self):
        self.messages = []
    
    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)
    
    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)
    
    def add_conversation(self, user_message: Dict, assistant_message: Dict):
        """Add a user-assistant conversation."""
        self.add_messages([user_message, assistant_message])
    
    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()
    
    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

## The Agent Tool Class

In [4]:
from pydantic import BaseModel, ValidationError
from typing import Callable, Type
from inspect import signature

class AgentTool:
    """Encapsulates a Python function with Pydantic validation."""
    def __init__(self, func: Callable, args_model: Type[BaseModel]):
        self.func = func
        self.args_model = args_model
        self.name = func.__name__
        self.description = func.__doc__ or self.args_schema.get('description', '')

    def to_openai_function_call_definition(self) -> dict:
        """Converts the tool to OpenAI Function Calling format."""
        schema_dict = self.args_schema
        description = schema_dict.pop("description", "")
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": description,
                "parameters": schema_dict
            }
        }

    @property
    def args_schema(self) -> dict:
        """Returns the tool's function argument schema as a dictionary."""
        schema = self.args_model.model_json_schema()
        schema.pop("title", None)
        return schema

    def validate_json_args(self, json_string: str) -> bool:
        """Validate JSON string using the Pydantic model."""
        try:
            validated_args = self.args_model.model_validate_json(json_string)
            return isinstance(validated_args, self.args_model)
        except ValidationError:
            return False

    def run(self, *args, **kwargs) -> Any:
        """Execute the function with validated arguments."""
        try:
            # Handle positional arguments by converting them to keyword arguments
            if args:
                sig = signature(self.func)
                arg_names = list(sig.parameters.keys())
                kwargs.update(dict(zip(arg_names, args)))

            # Validate arguments with the provided Pydantic schema
            validated_args = self.args_model(**kwargs)
            return self.func(**validated_args.model_dump())
        except ValidationError as e:
            raise ValueError(f"Argument validation failed for tool '{self.name}': {str(e)}")
        except Exception as e:
            raise ValueError(f"An error occurred during the execution of tool '{self.name}': {str(e)}")

    def __call__(self, *args, **kwargs) -> Any:
        """Allow the AgentTool instance to be called like a regular function."""
        return self.run(*args, **kwargs)

## The Tool Decorator

In [5]:
from typing import Callable, Optional, Type
from pydantic import BaseModel

def check_docstring(func: Callable):
    """Ensure the function has a docstring."""
    if not func.__doc__:
        raise ValueError(f"Function '{func.__name__}' must have a docstring.")

def Tool(func: Optional[Callable] = None, *, args_model: Type[BaseModel]) -> AgentTool:
    """Decorator to wrap a function with an AgentTool instance."""
    def decorator(f: Callable) -> AgentTool:
        check_docstring(f)
        return AgentTool(f, args_model=args_model)
    return decorator(func) if func else decorator


## The Agent Tool Executor Class

In [6]:
from typing import Any, Dict, List, Optional

class AgentToolExecutor:
    """Manages tool registration and execution."""
    
    def __init__(self, tools: Optional[List[AgentTool]] = None):
        self.tools: Dict[str, AgentTool] = {}
        if tools:
            for tool in tools:
                self.register_tool(tool)
    
    def register_tool(self, tool: AgentTool):
        """Registers a tool."""
        if tool.name in self.tools:
            raise ValueError(f"Tool '{tool.name}' is already registered.")
        self.tools[tool.name] = tool
      
    def execute(self, tool_name: str, *args, **kwargs) -> Any:
        """Executes a tool by name with given arguments."""
        tool = self.tools.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found.")
        try:
            return tool(*args, **kwargs)
        except Exception as e:
            raise ValueError(f"Error executing tool '{tool_name}': {e}") from e
    
    def get_tool_names(self) -> List[str]:
        """Returns a list of all registered tool names."""
        return list(self.tools.keys())
    
    def get_tool_details(self) -> str:
        """Returns details of all registered tools."""
        tools_info = [f"{tool.name}: {tool.description} Args schema: {tool.args_schema['properties']}" for tool in self.tools.values()]
        return '\n'.join(tools_info)

## Previous Agent Base Class

In [7]:
import logging
from typing import Dict, List, Optional

logger = logging.getLogger(__name__)

class Agent:
    """Integrates LLM client, tools, memory, and manages tool executions."""
    
    def __init__(self, llm_client, system_message: Dict[str, str], max_iterations: int = 10, tools: Optional[List[AgentTool]] = None):
        self.llm_client = llm_client
        self.executor = AgentToolExecutor()
        self.memory = ChatMessageMemory()
        self.system_message = system_message
        self.max_iterations = max_iterations
        self.tool_history = []
        self.function_calls = None
        
        # Register and convert tools
        if tools:
            for tool in tools:
                self.executor.register_tool(tool)
            self.function_calls = [tool.to_openai_function_call_definition() for tool in tools]

    def run(self, user_message: Dict[str, str]):
        """Generates responses, manages tool calls, and updates memory."""
        self.memory.add_message(user_message)

        for _ in range(self.max_iterations):
            chat_history = [self.system_message] + self.memory.get_messages() + self.tool_history
            response = self.llm_client.generate(chat_history, tools=self.function_calls)

            if self.parse_response(response):
                continue
            else:
                self.memory.add_message(response)
                self.tool_history = []
                return response

    def parse_response(self, response) -> bool:
        """Executes tool calls suggested by the LLM and updates tool history."""
        import json
        
        if response.tool_calls:
            self.tool_history.append(response)
            for tool in response.tool_calls:
                tool_name = tool.function.name
                tool_args = tool.function.arguments
                tool_args_dict = json.loads(tool_args)
                try:
                    logger.info(f"Executing {tool_name} with args: {tool_args}")
                    execution_results = self.executor.execute(tool_name, **tool_args_dict)
                    self.tool_history.append({
                        "role": "tool",
                        "tool_call_id": tool.id,
                        "name": tool_name,
                        "content": str(execution_results)
                    })
                except Exception as e:
                    raise ValueError(f"Execution error in tool '{tool_name}': {e}") from e
            return True
        return False

## Testing Tools in Prompt Only

In [8]:
# Setup client with environment variables
client = OpenAIChatCompletion(base_url='https://api.openai.com/v1', model='gpt-4')

# Define the system message
system_message = {"role": "system", "content": "You are a security assistant."}

# Initialize the Agent with the LLM client and system message
agent = Agent(llm_client=client, system_message=system_message)

USER_PROMPT = """
You have access to the following tools:

search(topic: str) - Use it to search information.
greet(person: str) - Use it to greet a person.

Respond to the user with the right tool and input whenever is needed.
When responding to the user, provide only ONE tool per $JSON_BLOB, as shown:

```
{{
    "name": $TOOL_NAME,
    "arguments": $INPUT
}}
```

User input: Hello, this is roberto!
"""

# Define a user message
user_message = {"role": "user", "content": USER_PROMPT}

# Generate a response using the agent
response = agent.run(user_message)
response

ChatCompletionMessage(content='{\n    "name": "greet",\n    "arguments": "roberto"\n}', role='assistant', function_call=None, tool_calls=None)

After receiving the response, pray 😅 and parse the message content:

In [9]:
import json

json.loads(response.content)

{'name': 'greet', 'arguments': 'roberto'}

### What about JSON Within Text? 🔍

Instructing an LLM to output JSON can result in mixed explanatory text and structured responses. When this happens, we can use regex patterns to extract the JSON embedded within the text.
For parsing complex nested JSON structures, Python's re module isn't sufficient since it lacks nested pattern matching support. The regex library, however, allows recursion with patterns like  r'\{(?:[^{}]|(?R))*\}', enabling effective parsing of deeply nested JSON data. (https://stackoverflow.com/a/54235803).

In [10]:
import regex
import json

def parse_nested_json(text):
    pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}') # Supports nested structures
    match = pattern.search(text)
    if match:
        try:
            return json.loads(match.group())
        except regex.JSONDecodeError:
            pass
    return None  # No valid JSON found or parsing error

## Implementing a Prompt Formatter 🗣️

### The String Prompt Template Class

In [17]:
import re

class StringPromptTemplate:
    """Handles dynamic prompt formatting for an AI agent."""

    def __init__(self, template: str):
        """Initializes the prompt template and extracts variables."""
        self.template = template
        self.variables = {}
        self.required_variables = self.extract_variables()

    def extract_variables(self):
        """Extracts placeholders from the template."""
        return set(re.findall(r'\{(.*?)\}', self.template))

    def update_variables(self, **kwargs):
        """Updates template variables."""
        self.variables.update(kwargs)
        self.required_variables -= set(kwargs.keys())

    def format_prompt(self, **kwargs):
        """Generates a formatted prompt and tracks remaining variables."""
        combined_variables = {**self.variables, **kwargs}
        self.required_variables -= set(kwargs.keys())
        return self.template.format(**combined_variables)

We can test the class by first updating the `USER_PROMPT` string, replacing hardcoded tools with a `{tool_details}` placeholder and initializing an instance.

In [18]:
USER_PROMPT = """
You have access to the following tools:

{tool_details}

Respond to the user with the right tool and input whenever is needed.
When responding to the user, provide only ONE tool per $JSON_BLOB, as shown:

```
{{
    "name": $TOOL_NAME,
    "arguments": $INPUT
}}
```

User input: What is the weather in Virginia?
"""
formatter = StringPromptTemplate(USER_PROMPT)

Then, define the previous tools with their respective Pydantic models.

In [19]:
# Python function arguments schema
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    location: str = Field(description="location to get weather for")

@Tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Get weather information based on location."""
    return f"{location}: 80F."

class JumpSchema(BaseModel):
    distance: str = Field(description="Distance for agent to jump")

@Tool(args_model=JumpSchema)
def jump(distance: str) -> str:
    """Jump a specific distance."""
    return f"I jumped the following distance {distance}"

tools = [get_weather,jump]

Get tool details and replace the `tool_details` placeholder.

In [23]:
# Get tool details
tools_info = [f"{tool.name}: {tool.description} Args schema: {tool.args_schema['properties']}" for tool in tools]
tool_details = '\n'.join(tools_info)

user_prompt_formatted = formatter.format_prompt(tool_details=tool_details)
print(user_prompt_formatted)


You have access to the following tools:

get_weather: Get weather information based on location. Args schema: {'location': {'description': 'location to get weather for', 'title': 'Location', 'type': 'string'}}
jump: Jump a specific distance. Args schema: {'distance': {'description': 'Distance for agent to jump', 'title': 'Distance', 'type': 'string'}}

Respond to the user with the right tool and input whenever is needed.
When responding to the user, provide only ONE tool per $JSON_BLOB, as shown:

```
{
    "name": $TOOL_NAME,
    "arguments": $INPUT
}
```

User input: What is the weather in Virginia?



### 🚦Assembly Line: Integrating Prompt Formatter with AI Agent

In [24]:
import logging
from typing import Dict, List, Optional

logger = logging.getLogger(__name__)

class Agent:
    """Integrates key components and manages tool executions."""
    
    def __init__(self, llm_client, system_message: Dict[str, str], max_iterations: int = 10, tools: Optional[List[AgentTool]] = None, prompt_template: StringPromptTemplate = None):
        self.llm_client = llm_client
        self.executor = AgentToolExecutor()
        self.memory = ChatMessageMemory()
        self.system_message = system_message
        self.max_iterations = max_iterations
        self.tool_history = []
        self.function_calls = None
        self.prompt_template = prompt_template

        if tools:
            for tool in tools:
                self.executor.register_tool(tool)
            self.function_calls = [tool.to_openai_function_call_definition() for tool in tools]

        tool_details = self.executor.get_tool_details()
        tool_names = ' or '.join(self.executor.get_tool_names())
        self.prompt_template.update_variables(
            system_message=self.system_message,
            tool_details=tool_details,
            tool_names=tool_names
        )

    def run(self, task: str):
        """Generates responses, manages tool calls, and updates memory."""
        self.memory.add_message({"role": "user", "content": task})

        for _ in range(self.max_iterations):
            chat_history = self.messages_to_string()
            formatted_message = self.prompt_template.format_prompt(chat_history=chat_history, user_input=task)
            messages = [{"role": "user", "content": formatted_message}]
            response = self.llm_client.generate(messages=messages)
            self.memory.add_conversation(user_message={"role": "user", "content": task}, assistant_message=response)
            return response

    def parse_response(self):
        """Parses LLM response."""
        pass
    
    def messages_to_string(self) -> str:
        """Converts messages to a string."""
        formatted_messages = []
        for message in self.memory.get_messages():
            formatted_messages.append(f"{message['role'].capitalize()}: {message['content']}")
        return "\n".join(formatted_messages)

## Enable Tool Execution v2 ⚒️

In [31]:
import logging
from typing import Dict, List, Optional

logger = logging.getLogger(__name__)

class Agent:
    """Integrates key components and manages tool executions."""
    
    def __init__(self, llm_client, system_message: Dict[str, str], max_iterations: int = 10, tools: Optional[List[AgentTool]] = None, prompt_template: StringPromptTemplate = None):
        self.llm_client = llm_client
        self.executor = AgentToolExecutor()
        self.memory = ChatMessageMemory()
        self.system_message = system_message
        self.max_iterations = max_iterations
        self.tool_history = []
        self.function_calls = None
        self.prompt_template = prompt_template

        if tools:
            for tool in tools:
                self.executor.register_tool(tool)
            self.function_calls = [tool.to_openai_function_call_definition() for tool in tools]

        tool_details = self.executor.get_tool_details()
        tool_names = ' or '.join(self.executor.get_tool_names())
        self.prompt_template.update_variables(
            system_message=self.system_message,
            tool_details=tool_details,
            tool_names=tool_names
        )

    def run(self, task: str):
        """Generates responses, manages tool calls, and updates memory."""
        self.memory.add_message({"role": "user", "content": task})

        for _ in range(self.max_iterations):
            chat_history = self.messages_to_string()
            formatted_message = self.prompt_template.format_prompt(chat_history=chat_history, user_input=task)
            messages = [{"role": "user", "content": formatted_message}]
            response = self.llm_client.generate(messages=messages)
            action_dict = self.parse_response(response)

            if action_dict:
                action_name = action_dict["name"]
                action_arguments = action_dict["arguments"]
                logger.info(f"Executing {action_name} with arguments {action_arguments}")
                execution_results = self.executor.execute(action_name, **action_arguments)
                return execution_results
            else:
                logger.info("Agent is responding directly.")
                self.memory.add_conversation(user_message={"role": "user", "content": task}, assistant_message=response)
                return response

    def parse_response(self, response: Dict):
        """Extracts tools or continues the conversation."""
        import regex, json

        pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')  # Supports nested structures
        message_content = response.content
        match = pattern.search(message_content)
        if match:
            action_content = match.group()
            try:
                action_dict = json.loads(action_content.strip())
                return action_dict
            except json.JSONDecodeError:
                raise ValueError("Invalid JSON in action content")
        return None
    
    def messages_to_string(self) -> str:
        """Converts messages to a string."""
        formatted_messages = []
        for message in self.memory.get_messages():
            formatted_messages.append(f"{message['role'].capitalize()}: {message['content']}")
        return "\n".join(formatted_messages)

You can test this new agent mode with the following prompt template:

In [32]:
STRING_PROMPT_TEMPLATE = """
{system_message}. You have access to the following tools:

{tool_details}

Respond to the the user with the right tool and input whenever is needed.

Valid tool name values: {tool_names}.

When responding to the user, provide only ONE tool per $JSON_BLOB, as shown:

```
{{
    "name": $TOOL_NAME,
    "arguments": $INPUT
}}
```

Previous conversation history:
{chat_history}

New conversation:
{user_input}
"""

Initialize the prompt template as a `StringPromptTemplate`

In [33]:
prompt_template = StringPromptTemplate(STRING_PROMPT_TEMPLATE)

Initialize agent with an LLM client, system message, tools and prompt template.

In [34]:
# Initialize LLM client
client = OpenAIChatCompletion(base_url='https://api.openai.com/v1', model='gpt-4')

# Define the system message
system_message = {"role": "system", "content": "You are a security assistant."}

# Initialize the Agent with the LLM client and system message
agent = Agent(llm_client=client, system_message=system_message, tools=tools, prompt_template=prompt_template)

Finally, run the agent:

In [35]:
agent.run("What is the weather in New York?")

'New York: 80F.'

## ReAct Prompting

### Define ReAct Prompt

In [49]:
# References:
# https://smith.langchain.com/hub/hwchase17/react-chat
# https://smith.langchain.com/hub/hwchase17/structured-chat-agent

STRING_PROMPT_TEMPLATE = """
{system_message}. You have access to the following tools:

{tool_details}

Respond to the the user with the right tool and input whenever is needed.
Use a json blob to specify a tool by providing a name key (tool name) and an arguments key (tool input).

Valid "action" values:  {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:
```
{{
    "name": $TOOL_NAME,
    "arguments": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: reasoning about previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond as final answer. Using tool to provide final answer
Final Answer: Final response to human

Remember to ALWAYS respond with a valid json blob of a single action when a tool MUST be used.
Use tools if necessary. Respond directly if appropriate.
Format is Action:```$JSON_BLOB```then Observation'

Previous conversation history:
{chat_history}

New conversation:
Question: {user_input}
{react_loop}
"""
prompt_template = StringPromptTemplate(STRING_PROMPT_TEMPLATE)

### 🚦Assembly Line: Integrating ReAct Prompt with AI Agent 🤖

In [50]:
import logging
from typing import Dict, List, Optional
from pydantic import BaseModel
from typing import List, Dict

logger = logging.getLogger(__name__)

class Agent:
    """
    Basic Agent class responsible for integrating key components such as the LLM client, tools, memory, and managing tool executions.
    """
    def __init__(self, llm_client, system_message: Dict[str, str], max_iterations: int = 10, tools: Optional[List[AgentTool]] = None, prompt_template : StringPromptTemplate=None):
        self.llm_client = llm_client
        self.executor = AgentToolExecutor()
        self.memory = ChatMessageMemory()
        self.system_message = system_message
        self.max_iterations = max_iterations
        self.tool_history = []
        self.function_calls = None
        self.prompt_template = prompt_template  # Instance of StringPromptTemplate or similar
        
        # Register each tool passed to the Agent using the executor
        if tools:
            for tool in tools:
                # Register Agent Tools
                self.executor.register_tool(tool)
                # Convert Agent Tools
                self.function_calls = [tool.to_openai_function_call_definition() for tool in tools]
        
        # Pre-fill the prompt template with initial variables
        tool_details=self.executor.get_tool_details(),
        tool_names=' or '.join(self.executor.get_tool_names())
        self.prompt_template.update_variables(
            system_message=self.system_message,
            tool_details=tool_details,
            tool_names=tool_names
        )

    def run(self, task:str):
        # Get Chat History
        chat_history = self.messages_to_string()
        # Initialize ReAct Loop
        react_loop = ""

        # Showing Initial Task
        print(f"Question: {task}")

        for _ in range(self.max_iterations):
            # Generate a dynamic prompt using variables
            formatted_message = self.prompt_template.format_prompt(chat_history=chat_history,user_input=task,react_loop=react_loop)
            # Define everything as a user message
            messages = [{"role":"user", "content": formatted_message}]
            
            # Instruct LLM to choose the right tool and respond with a structured output
            response = self.llm_client.generate(messages=messages,stop=["\nObservation:"],)
            # Parse response and extract tools if any
            action_dict = self.parse_response(response)

            if action_dict:
                action_name = action_dict["name"]
                action_arguments = action_dict["arguments"]
                
                logger.info(f"Executing {action_name} with arguments {action_arguments}")
                execution_results = self.executor.execute(action_name, **action_arguments)
                
                current_iteration = f"\nThought: {response.content}\nObservation: {execution_results}"
                react_loop += current_iteration
                print(current_iteration)
            else:
                message_content = response.content
                print(message_content)
                if 'final answer' in str(message_content).lower():
                    final_message = str(message_content).lower().split("final answer:")[-1].strip()
                    response = {
                        "role": "assistant",
                        "content": final_message
                    }
                
                logger.info("Agent is responding directly.")
                self.memory.add_conversation(
                    user_message={"role":"user","content": task},
                    assistant_message=response
                )
                return response    

    def parse_response(self, response: Dict):
        """
        Extracts tools or continues the conversation.
        """
        import regex, json

        pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}') # Supports nested structures
        message_content = response.content
        match = pattern.search(message_content)
        if match:
            action_content = match.group()
            try:
                action_dict = json.loads(action_content.strip())
                return action_dict
            except regex.JSONDecodeError:
                raise ValueError("Invalid JSON in action content")
    
    def messages_to_string(self) -> str:
        """
        Converts a list of message objects or dictionaries into a multi-line string representation.
        """
        formatted_messages = []

        for message in self.memory.get_messages():
            if isinstance(message, BaseModel):
                message = message.model_dump()
            formatted_messages.append(f"{message["role"].capitalize()}: {message["content"]}")
        return "\n".join(formatted_messages)

In [51]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(
        self,
        messages: List[str],
        tools: List[Dict[str, Any]] = None,
        stop: List[str] = None,
        **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {
            'messages': messages,
            'model': self.model,
            'tools': tools,
            'stop': stop,
            **kwargs
        }
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

I updated the LLM Client class to use the stop parameter in [OpenAI's Chat Completion API](https://platform.openai.com/docs/api-reference/chat/create), stopping token generation at `\nObservation`:

In [52]:
# Initialize LLM client
client = OpenAIChatCompletion(base_url='https://api.openai.com/v1', model='gpt-4')

# Define the system message
system_message = {"role": "system", "content": "You are a security assistant."}

# Initialize the Agent with the LLM client and system message
agent = Agent(llm_client=client, system_message=system_message, tools=tools, prompt_template=prompt_template)

In [53]:
agent.run("What is the weather in New York?")

Question: What is the weather in New York?

Thought: Thought: The user is asking for the weather information in New York. I will use the "get_weather" tool to provide this information.

Action:
```
{
    "name": "get_weather",
    "arguments": {"location": "New York"}
}
```
Observation: New York: 80F.
Thought: I observed that the tool successfully retrieved the current temperature in New York, to which is 80F.  Now I can respond to the user's query.

Final Answer: The current weather in New York is 80F.


{'role': 'assistant', 'content': 'the current weather in new york is 80f.'}

In [54]:
agent.memory.get_messages()

[{'role': 'user', 'content': 'What is the weather in New York?'},
 {'role': 'assistant', 'content': 'the current weather in new york is 80f.'}]

In [55]:
agent.memory.reset_memory()

In [56]:
agent.run("What is the weather in New York, Paris, and Virginia?")

Question: What is the weather in New York, Paris, and Virginia?

Thought: Thought: The user wants to know the weather in three different locations: New York, Paris, and Virginia. I have a tool for fetching the weather information, but it requires a string for a single location as an argument. I will need to handle each location one by one before providing a final response.

Action:
```
{
    "name": "get_weather",
    "arguments": {"location": "New York"}
}
```
Observation: New York: 80F.

Thought: Thought: I have gathered the weather information for New York. Now I need to get the weather for Paris.

Action:
```
{
    "name": "get_weather",
    "arguments": {"location": "Paris"}
}
```
Observation: Paris: 80F.

Thought: Thought: I have gathered the weather information for New York and Paris. Now I need to get the weather for Virginia.

Action:
```
{
    "name": "get_weather",
    "arguments": {"location": "Virginia"}
}
```
Observation: Virginia: 80F.
Thought: Now I have the weather inf

{'role': 'assistant',
 'content': 'the current weather is 80f in new york, 80f in paris, and 80f in virginia.'}